In [1]:
import pandas as pd
import numpy as np
import requests
import re
import os
from time import sleep
from tqdm import tqdm
from functools import reduce

import datetime

In [18]:
BASE_GOOGLE_FINANCE_URL = "https://api.polygon.io/v2/aggs/ticker/{TICKER}/range/{INTERVAL}/" +\
                          "{RANGE}/{FROM_DATE}/{TO_DATE}?limit=50000&apiKey=MNxlbyNXPpKmAki1jcScqmXxzPvdXWYT"

# TOKENS = ["AAPL", "GOOG", "MSFT", "TSLA", "NVDA"]
TOKENS = ["AAPL"]

TIME_INTERVAL = 15
TIME_UNITS = "minute"
FROM_DATE = pd.Timestamp("2019-02-18").date()
TO_DATE = pd.Timestamp("2022-03-28").date()

DATASET_NAME = "AAPL_prices"

TRAIN_RESULT_PATH = f"prices/{DATASET_NAME}_{TIME_INTERVAL}{TIME_UNITS}_train.csv"
TEST_RESULT_PATH = f"prices/{DATASET_NAME}_{TIME_INTERVAL}{TIME_UNITS}_test.csv"

In [4]:
def get_url(token: str, interval_len: int, interval: str, from_date: datetime.date, to_date: datetime.date):
    """
    Build url for Google Finance API
    """
    url = re.sub("{TICKER}", token, BASE_GOOGLE_FINANCE_URL)
    url = re.sub("{INTERVAL}", str(interval_len), url)
    url = re.sub("{RANGE}", interval, url)
    url = re.sub("{FROM_DATE}", str(from_date), url)
    url = re.sub("{TO_DATE}", str(to_date), url)
    return url

In [5]:
def get_data(token: str, interval_len: int, interval: str, from_date: datetime.date, to_date: datetime.date):
    TIMEOUT = 20
    current_date = from_date
    previous_date = current_date
    dfs = []
    while current_date <= to_date:
        request_url = get_url(token, interval_len, interval, current_date, to_date)
        data = requests.get(url=request_url)
        if data.json()["status"] == "ERROR":
            print("Waiting for", TIMEOUT, "seconds")
            for _ in tqdm(range(TIMEOUT)):
                sleep(1)
        else:
            try:
                current_df = pd.DataFrame(data.json()["results"])
            except:
                pass
            current_date = (pd.Timestamp(current_df["t"].max(), unit="ms") + pd.DateOffset(days=1)).date()
            print("Current date:", current_date)
            print("Get DataFrame of shape", current_df.shape)
            dfs.append(current_df)
    result_df = pd.concat(dfs)
    result_df.drop_duplicates(inplace=True)
    return result_df

In [6]:
dfs = []
for token in TOKENS:
    data = get_data(
        token=token,
        interval_len=TIME_INTERVAL,
        interval=TIME_UNITS,
        from_date=FROM_DATE, 
        to_date=TO_DATE
    )

    data["t"] = data["t"].apply(lambda row: pd.Timestamp(row, unit="ms"))
    data["ticker"] = token
    dfs.append(data)

Current date: 2020-07-15
Get DataFrame of shape (4390, 8)
Current date: 2020-10-08
Get DataFrame of shape (3809, 8)
Current date: 2021-01-01
Get DataFrame of shape (3662, 8)
Current date: 2021-03-26
Get DataFrame of shape (3616, 8)
Current date: 2021-06-24
Get DataFrame of shape (3939, 8)
Waiting for 20 seconds


100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [00:20<00:00,  1.01s/it]


Waiting for 20 seconds


100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [00:20<00:00,  1.01s/it]


Current date: 2021-09-24
Get DataFrame of shape (4041, 8)
Current date: 2021-12-22
Get DataFrame of shape (3909, 8)
Current date: 2022-03-22
Get DataFrame of shape (3853, 8)
Current date: 2022-03-29
Get DataFrame of shape (319, 8)


In [24]:
RENAME_COLUMNS = {
    "v": "volume",
    "o": "open",
    "c": "close",
    "h": "high",
    "l": "low",
    "t": "date"
}
KEEP_COLUMNS = ["volume", "open", "close", "high", "low", "date", "ticker"]

result_df = pd.concat(dfs).rename(RENAME_COLUMNS, axis=1)[KEEP_COLUMNS].reset_index(drop=True)
result_df.sample(10)

,volume,open,close,high,low,date,ticker
13585,34107.0,135.2500,135.2400,135.2700,135.2300,2021-02-11 00:00:00,AAPL
27003,1521931.0,175.4500,175.6099,175.7400,175.4000,2021-12-13 21:30:00,AAPL
26261,4983.0,161.8900,161.9000,161.9200,161.8800,2021-11-24 22:15:00,AAPL
29811,129425.0,172.2401,172.0600,172.3000,171.8800,2022-02-16 13:45:00,AAPL
22083,12099.0,149.7100,149.7100,149.7300,149.7000,2021-08-23 22:00:00,AAPL
13572,7170570.0,135.0100,135.3600,135.4200,134.9500,2021-02-10 20:45:00,AAPL
17849,4925.0,123.6000,123.6100,123.7100,123.6000,2021-05-19 09:15:00,AAPL
149,236676.0,65.5900,65.4175,65.7500,65.4067,2020-04-08 12:43:00,AAPL
7213,3270012.0,114.1100,114.0943,114.3200,113.9600,2020-09-16 15:30:00,AAPL
2590,5952680.0,80.9775,81.1242,81.1288,80.9200,2020-06-03 13:43:00,AAPL


In [25]:
TEST_RATIO = 0.2
test_size = int(result_df.shape[0] * TEST_RATIO)
data_train = result_df.iloc[:-test_size]
data_test = result_df.iloc[-test_size:]

data_train.to_csv(TRAIN_RESULT_PATH)
data_test.to_csv(TEST_RESULT_PATH)